In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10

In [2]:
hat_data_df = pd.read_csv("Full Hat Data.csv")
hat_data_df =hat_data_df[hat_data_df.columns.drop("Unnamed: 0")]

## Getting the Survey Data

In [3]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1e6T2zlrZ1kaXXX-4b75b7vrwH51kh4SReAia2A0OFn8'
SAMPLE_RANGE_NAME = 'B1:DO'

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
ratings_df = pd.DataFrame(result["values"])

In [4]:
ratings_df = ratings_df.rename(columns=ratings_df.iloc[0]).drop([0])
ratings_df_s = ratings_df.T

In [5]:
ratings_df_s[ratings_df_s.columns] = ratings_df_s[ratings_df_s.columns].apply(pd.to_numeric)
mean_values = ratings_df_s.mean(axis = 1).sort_values(ascending = False)
community_rating_df = pd.DataFrame(mean_values).reset_index().rename(columns = {"index" : "Effect", 0: "Effect Community Rating"})

In [6]:
effect_data_df = pd.read_csv("Full Effect Data.csv")
effect_data_df = effect_data_df[effect_data_df.columns.drop("Unnamed: 0")]
effect_data_df = effect_data_df.merge(community_rating_df,on="Effect",how = "left")
effect_data_df

,Effect,Generation,Id,Effect Community Rating
0,Burning Flames,First Gen,13,4.575758
1,Circling Heart,First Gen,19,3.060606
2,Circling Peace Sign,First Gen,18,2.242424
3,Circling TF Logo,First Gen,11,2.393939
4,Green Confetti,First Gen,6,1.393939
...,...,...,...,...
117,Magnetic Hat Protector,Invasion,95,2.212121
118,Voltaic Hat Protector,Invasion,96,2.151515
119,Galactic Codex,Invasion,97,1.545455
120,Ancient Codex,Invasion,98,1.575758


In [7]:
taunt_effects = effect_data_df[effect_data_df["Id"] >= 3000]
taunt_effects

,Effect,Generation,Id,Effect Community Rating
52,Ghastly Ghosts,Halloween 2014,3012,2.909091
53,Haunted Phantasm,Halloween 2014,3011,2.969697
65,Spectral Swirl,Halloween 2016,3014,2.909091
66,Hellish Inferno,Halloween 2016,3013,2.909091
67,Infernal Smoke,Halloween 2016,3016,3.090909
...,...,...,...,...
100,Midnight Whirlwind,Love & War,3008,0.545455
101,Screaming Tiger,Love & War,3006,2.454545
102,Showstopper,Love & War,3001,2.242424
103,Silver Cyclone,Love & War,3009,0.787879


In [8]:
hat_effects = effect_data_df[effect_data_df["Id"] < 200]
hat_effects

,Effect,Generation,Id,Effect Community Rating
0,Burning Flames,First Gen,13,4.575758
1,Circling Heart,First Gen,19,3.060606
2,Circling Peace Sign,First Gen,18,2.242424
3,Circling TF Logo,First Gen,11,2.393939
4,Green Confetti,First Gen,6,1.393939
...,...,...,...,...
117,Magnetic Hat Protector,Invasion,95,2.212121
118,Voltaic Hat Protector,Invasion,96,2.151515
119,Galactic Codex,Invasion,97,1.545455
120,Ancient Codex,Invasion,98,1.575758


## Getting the JSON Pricing Data From Backpack.tf

In [9]:
import json
import requests

bptf_key = "58324a43c440450c7d4e2679"

resp = requests.get("https://backpack.tf/api/IGetPrices/v4?key=" + bptf_key)
json_result = resp.json()

In [10]:
items_dict = json_result["response"]["items"]

unusual_items = np.array(hat_data_df[["Name","Item Type"]])
h_effects = np.array(hat_effects[["Effect","Id"]])
t_effects = np.array(taunt_effects[["Effect","Id"]])

final_array = []

for item in items_dict:
    
    for i in range(0,len(unusual_items)):
        
        item_to_compare = item
        if item.startswith("Taunt: "):
            item_to_compare = item[7:]
        
        if item_to_compare == unusual_items[i][0]:
        
            item_type = unusual_items[i][1]
            #print(item,item_type)
            
            item_json = items_dict[item]["prices"].get("5")["Tradable"]["Craftable"]
            
            if item_type == "Cosmetic":
                for pair in h_effects:
                    
                    effect_name = pair[0]
                    effect_id = pair[1]
                    price_final = np.nan
                    last_update = np.nan
                    
                    prices_dict = item_json.get(str(effect_id))
                    
                    if prices_dict != None:
                        
                        if prices_dict["last_update"] > 1427893200: # a price newer than April 1, 2015
                            last_update = prices_dict["last_update"]
                            
                        
                            if len(prices_dict) == 5:

                                price_low = prices_dict["value"]
                                price_high = prices_dict["value_high"]

                                price_final = float(price_low + price_high)/2

                            else:

                                price_final = prices_dict["value"]

                        
                    to_add = [item_to_compare,effect_name,price_final,last_update]
                    final_array.append(to_add)
                    
            elif item_type == "Taunt":
                for pair in t_effects:
                    
                    effect_name = pair[0]
                    effect_id = pair[1]
                    price_final = np.nan
                    last_update = np.nan
                    
                    prices_dict = item_json.get(str(effect_id))
                    
                    if prices_dict != None:
                        
                        if prices_dict["last_update"] > 1427893200: # a price newer than April 1, 2015
                            last_update = prices_dict["last_update"]
                        
                            if len(prices_dict) == 5:

                                price_low = prices_dict["value"]
                                price_high = prices_dict["value_high"]

                                price_final = float(price_low + price_high)/2

                            else:

                                price_final = prices_dict["value"]
                    
                    to_add = [item_to_compare,effect_name,price_final,last_update]
                    final_array.append(to_add)
                    
prices_df = pd.DataFrame(final_array).rename(columns = {0:"Name",1:"Effect",2:"Price",3:"Last Price Update"})
prices_df

,Name,Effect,Price,Last Price Update
0,Dread Knot,Burning Flames,30.0,1.530629e+09
1,Dread Knot,Circling Heart,NaN,NaN
2,Dread Knot,Circling Peace Sign,10.0,1.499052e+09
3,Dread Knot,Circling TF Logo,9.0,1.439780e+09
4,Dread Knot,Green Confetti,9.0,1.514740e+09
...,...,...,...,...
41373,Snowcapped,Magnetic Hat Protector,NaN,NaN
41374,Snowcapped,Voltaic Hat Protector,NaN,NaN
41375,Snowcapped,Galactic Codex,NaN,NaN
41376,Snowcapped,Ancient Codex,NaN,NaN


In [11]:
temp_df = prices_df.merge(hat_data_df,on = "Name", how = "left")

In [12]:
unusuals_df = temp_df.merge(effect_data_df,on = "Effect", how = "left")

In [13]:
import time
import calendar
unusuals_df["Crate Cosmetic"] = unusuals_df.apply(
    lambda row: True if row["Grade"] == "No Grade" and row["Item Type"] == "Cosmetic" else False,
    axis=1
) #(2019 - time.gmtime(row["Last Price Update"])[0]) * 12 + 
unusuals_df["Months Since Last Price Update"] = unusuals_df.apply(
    lambda row: -1 if pd.isnull(row["Last Price Update"]) else (time.gmtime(calendar.timegm(time.gmtime()) - row["Last Price Update"])[1] + 
    (time.gmtime(calendar.timegm(time.gmtime()) - row["Last Price Update"])[0] - 1970) * 12),
    axis=1
)

In [14]:
unusuals_df = unusuals_df[unusuals_df.columns.drop(["Id","Last Price Update"])]

In [15]:
median_effect_prices = pd.DataFrame(unusuals_df.groupby("Effect")["Price"].median().sort_values(ascending = False)).reset_index().rename(columns = {"Price" : "Effect Median Price"})
median_effect_prices

,Effect,Effect Median Price
0,Nebula,180.000
1,Abduction,153.750
2,Spellbound,140.000
3,Bonzo The All-Gnawing,138.500
4,Arcana,120.000
...,...,...
113,Mega Strike,7.375
114,Skill Gotten Gains,7.250
115,'72,7.000
116,Midnight Whirlwind,6.250


In [16]:
median_hat_prices = pd.DataFrame(unusuals_df.groupby("Name")["Price"].median().sort_values(ascending = False)).reset_index().rename(columns = {"Price" : "Hat Median Price"})
median_hat_prices

,Name,Hat Median Price
0,Coffin Kit,2210.00
1,Coldfront Curbstompers,1750.00
2,Sleeveless in Siberia,892.00
3,Potassium Bonnett,645.00
4,Antlers,482.50
...,...,...
449,Results Are In,6.25
450,Spent Well Spirits,6.00
451,Battin' a Thousand,5.50
452,I See You,5.00


In [17]:
unusuals_df = unusuals_df.merge(median_effect_prices,on= "Effect",how = "left")
unusuals_df = unusuals_df.merge(median_hat_prices, on = "Name",how = "left")
unusuals_df

,Name,Effect,Price,Class,Item Type,Equip Region,Grade,Unboxed From,Robo,Generation,Effect Community Rating,Crate Cosmetic,Months Since Last Price Update,Effect Median Price,Hat Median Price
0,Dread Knot,Burning Flames,30.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,4.575758,True,9,104.00,14.0
1,Dread Knot,Circling Heart,NaN,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,3.060606,True,-1,36.25,14.0
2,Dread Knot,Circling Peace Sign,10.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,2.242424,True,21,22.00,14.0
3,Dread Knot,Circling TF Logo,9.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,2.393939,True,44,21.50,14.0
4,Dread Knot,Green Confetti,9.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,1.393939,True,15,17.00,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41373,Snowcapped,Magnetic Hat Protector,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,2.212121,False,-1,20.50,22.0
41374,Snowcapped,Voltaic Hat Protector,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,2.151515,False,-1,52.50,22.0
41375,Snowcapped,Galactic Codex,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,1.545455,False,-1,36.75,22.0
41376,Snowcapped,Ancient Codex,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,1.575758,False,-1,19.50,22.0


In [18]:
unusuals_df["Hat Rating"] = (unusuals_df["Price"] - unusuals_df["Effect Median Price"]) / 10
median_hat_ratings = pd.DataFrame(unusuals_df.groupby("Name")["Hat Rating"].median().sort_values(ascending = False)).reset_index().rename(columns = {"Hat Rating" : "Median Hat Rating"})
median_hat_ratings

,Name,Median Hat Rating
0,Coffin Kit,217.800
1,Coldfront Curbstompers,170.000
2,Sleeveless in Siberia,84.950
3,Potassium Bonnett,62.675
4,Antlers,45.600
...,...,...
449,Magnificent Mongolian,-1.900
450,Commando Elite,-2.050
451,Tank Top,-2.400
452,Cool Capuchon,-2.450


In [19]:
unusuals_df["Effect Rating"] = (unusuals_df["Price"] - unusuals_df["Hat Median Price"]) / 10
median_effect_ratings = pd.DataFrame(unusuals_df.groupby("Effect")["Effect Rating"].median().sort_values(ascending = False)).reset_index().rename(columns = {"Effect Rating" : "Median Effect Rating"})
median_effect_ratings

,Effect,Median Effect Rating
0,Nebula,14.5000
1,Spellbound,10.7500
2,Bonzo The All-Gnawing,10.1500
3,Arcana,8.5000
4,The Eldritch Opening,8.0125
...,...,...
113,Massed Flies,-1.7000
114,Aces High,-1.7000
115,Bubbling,-1.7250
116,Orbiting Planets,-1.7500


In [20]:
unusuals_df = unusuals_df.merge(median_effect_ratings,on= "Effect",how = "left")
unusuals_df = unusuals_df.merge(median_hat_ratings, on = "Name",how = "left")
unusuals_df

,Name,Effect,Price,Class,Item Type,Equip Region,Grade,Unboxed From,Robo,Generation,Effect Community Rating,Crate Cosmetic,Months Since Last Price Update,Effect Median Price,Hat Median Price,Hat Rating,Effect Rating,Median Effect Rating,Median Hat Rating
0,Dread Knot,Burning Flames,30.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,4.575758,True,9,104.00,14.0,-7.40,1.6,6.7375,-1.6000
1,Dread Knot,Circling Heart,NaN,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,3.060606,True,-1,36.25,14.0,NaN,NaN,0.4500,-1.6000
2,Dread Knot,Circling Peace Sign,10.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,2.242424,True,21,22.00,14.0,-1.20,-0.4,-0.8500,-1.6000
3,Dread Knot,Circling TF Logo,9.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,2.393939,True,44,21.50,14.0,-1.25,-0.5,-0.9125,-1.6000
4,Dread Knot,Green Confetti,9.0,Heavy,Cosmetic,Regular,No Grade,Old Series,False,First Gen,1.393939,True,15,17.00,14.0,-0.80,-0.5,-1.4000,-1.6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41373,Snowcapped,Magnetic Hat Protector,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,2.212121,False,-1,20.50,22.0,NaN,NaN,-0.9000,-0.1625
41374,Snowcapped,Voltaic Hat Protector,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,2.151515,False,-1,52.50,22.0,NaN,NaN,-1.1750,-0.1625
41375,Snowcapped,Galactic Codex,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,1.545455,False,-1,36.75,22.0,NaN,NaN,-1.1000,-0.1625
41376,Snowcapped,Ancient Codex,NaN,Medic,Cosmetic,Regular,Mercenary,Winter 2018 Cosmetic,False,Invasion,1.575758,False,-1,19.50,22.0,NaN,NaN,-1.2500,-0.1625


In [21]:
unusuals_df = unusuals_df[~unusuals_df["Median Hat Rating"].isnull()]

In [22]:
unusuals_train_df = unusuals_df[~unusuals_df["Price"].isnull()]
unusuals_predict_df = unusuals_df[unusuals_df["Price"].isnull()]

In [23]:
unusuals_df.to_csv("CompleteHatAndPricingData.csv")